In [1]:
from joblib import Parallel, delayed
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
from scipy.signal import convolve2d
from scipy.signal import convolve
import copy
import json
import time as tm
from numba import jit, prange
import random
from copy import deepcopy

In [ ]:
# Time and dimensions
side_len = 750 #microns
box_height = 300 #microns
cell_size = 5 #microns
cell_to_diff_ratio = 10
density = 10 #mm^-2

dt_diff = 1 #s
dt_cell = 0.1 #hour
dt_cell_to_diff_ratio = 360
t_final = 24 #hours

nsteps = int(t_final/dt_cell)
time = np.arange(nsteps)/dt_diff + dt_diff

D = 360 #um^2/s
DS = D/np.square(cell_size*cell_to_diff_ratio) #cell_side^2/s
#print(DS)

nx = ny = int(side_len/(cell_size*cell_to_diff_ratio))
nz = int(box_height/(cell_size*cell_to_diff_ratio))
S1 = np.zeros((nx, ny, nz))
S2 = np.zeros((nx, ny, nz))
D1, D2 = [DS,DS]

In [ ]:
class Cell:
    """Hold the properties and variables for strains."""
    def __init__(self,
                 x: int,
                 y: int,
                 z: int,
                 dependencies: np.ndarray,
                 d: float,
                 r: float,
                 n_nutrients: int,
                 alphas: list,
                 typ: int
                ):
        
        self.x = x
        self.y = y
        self.z = z
        
        self.stochastic_growth = True
        if any(dependencies[:,0]):
            self.stochastic_growth = False
        
        self.dependencies = dependencies
        self.is_alive = True
        self.quiescence_flag = False
        self.death_rate = d
        self.reproduction_rate = r
        self.alphas = alphas
        self.typ = typ
        
        self.S_internal = [None]*n_nutrients
        #print(n_nutrients)
        #print(np.shape(self.S_internal))
        for ii in range(n_nutrients):
            #print(ii)
            #print(random.random()*alphas[ii]*dependencies[ii,0])
            self.S_internal[ii] = random.random()*alphas[ii]*dependencies[ii,0]
        
    
    def process_nutrient_uptake(self,n_idx,ammnt):
        self.S_internal[n_idx] += ammnt
        return None
        
    def process_nutrient_release(self,n_idx,ammnt):
        #This will be useful later when we think about pathways, but for now it is just a placeholder
        return None
    
    def create_daughter(self, x, y, z):
        """
        Create a daughter cell at the specified position
        Should return a new Cell instance with:
        - Same dependencies and parameters as parent
        - New position (x, y, z)
        - Zero internal nutrients
        """
        daughter = Cell(x,y,z,
                        self.dependencies,
                        self.death_rate,
                        self.reproduction_rate,
                        n_nutrients,
                        [0,0],
                        self.typ
                       )
        return daughter

In [ ]:
def initialize_population(side_len,cell_size,n_strains,density,n_nutrients,dependencies,alphas,d):
    
    n_cells = int(density*cell_size**2*side_len**2/1e6)
    cell_sides = side_len/cell_size
    
    #seed in a box of width side_len/4
    width = int(cell_sides/4)
    offset = int(cell_sides/2 - cell_sides/8)
    
    initial_cells = [None]*n_cells
    
    for nn in range(n_cells):
        x = int(width*np.random.rand()) + offset
        y = int(width*np.random.rand()) + offset
        z = 0 #int(width*np.random.rand())
        
        #pick which strain this position will be
        strain_identifier = int(np.floor((np.random.rand()-0.5)*n_strains + 1))
            
        initial_cells[nn] = Cell(x,y,z,
                        dependencies[strain_identifier,:,:],
                        d[strain_identifier],
                        r[strain_identifier],
                        n_nutrients,
                        alphas,
                        int(strain_identifier+1)
                       )
        
    return initial_cells

In [ ]:
"""
nutrient initialization 
"""
#We are going to do something a little wild here. Because all of our CoSMO strains have the same genetic background
#outside of the specific mutations, we are going to assume that the effect of growth of the various nutrients on
#different strains is the same. That is, we can define how a nutrient affects cell growth and apply those same 
#rates to every other cell that depends on that nutrient. In this manner, we can write out all of the rates
#for each nutrient once and then simply apply those constraints to a cell's growth by defining its dependencies. 
#This will further allow us to flexibly modulate the various interdependencies of different cell types.

#Lys
alphaL = 5.4 #nutrient required for reproduction (fmole)
rL = 0.51 #Maximum growth rate (hr^-1)
vmL = alphaL*rL #maximum uptake rate (fmole/hr)
"""
060125 The dt for diffusion is 1 second, so to ease calculations, we will put all of the parameters that are 
updated at the same time as diffusion in terms of seconds.
"""
vmL_s = vmL/(60^2) #maximum uptake rate (fmole/s)
KL = 2.1e6 #Monod constant (fmole/ml)
gammaL = 0.4 #release rate (fmole/(cell*hr))
gammaL_s = gammaL/(60^2) #release rate (fmole/(cell*s))
dL = 0.021 #death rate (hr^-1)

#Ade
alphaA = 3.1
rA = 0.44
vmA = alphaA*rA
vmA_s = vmA/(60^2)
KA = 1.3e6
gammaA = 0.26
gammaA_s = gammaA/(60^2)
dA = 0.015 #death rate (hr^-1). Should this be an order of magnitude higher?


alphas = [alphaL,alphaA]
vm_list = [vmL,vmA]
release_rates = [gammaL,gammaA]
km_list = [KL,KA]
d = [dL,dA]
r = [rL,rA]

In [ ]:
filepath = 'cosmo_claude_test_'

n_strains = 2
n_nutrients = 2

#The dependencies of strain the strains on each nutrient. Within each strain and nutrient, COLUMN 1 is UPTAKE, 
#COLUMN 2 is RELEASE. 1 means they do that function, 0 means they do not
dependencies = np.zeros((n_strains,n_nutrients,2))


#This initialization is for classic CoSMO type crossfeeding.
""""""
dependencies[0,0,0] = 1 #strain 1, uptake nutrient 1
dependencies[0,1,1] = 1 #strain 1, release nutrient 2

dependencies[1,1,0] = 1 #strain 2, uptake nutrient 2
dependencies[1,0,1] = 1 #strain 2, release nutrient 1

In [ ]:
def convolution_laplacian_3D(M: np.ndarray) -> np.ndarray:
    #Make a big matrix to incorporate boundary conditions
    #for this implementation, we will have periodic conditions in x and y and no flux in Z.
    #This involves literally just adding rows such that the boundary conditions are satisfied.
    #For periodic, the padded layer will be the value from the opposite side of the matrix,
    #for no flux, the padded layer (-1) equals the (1) layer such that the central difference around (0) is 0
    shp = np.shape(M)
    
    #make the padded matrix
    M_padded = np.zeros((shp[0]+2,shp[1]+2,shp[2]+2))
    
    #put the original matrix inside
    M_padded[1:-1,1:-1,1:-1] = M.copy()
    
    #pad the edges, starting with Z
    M_padded[:,:,0] = M_padded[:,:,2]
    M_padded[:,:,-1] = M_padded[:,:,-3]
    
    #pad the edges, X direction
    M_padded[0,:,:] = M_padded[-2,:,:]
    M_padded[-1,:,:] = M_padded[1,:,:]
    
    #pad the edges, Y direction
    M_padded[:,0,:] = M_padded[:,-2,:]
    M_padded[:,-1,:] = M_padded[:,1,:]
    
    """
    """
    #using the 27 point stencil
    #k = 1/26*[[[2,3,2],[3,6,3],[2,3,2]],[[3,6,3],[6,-88,6],[3,6,3]],[[2,3,2],[3,6,3],[2,3,2]]]
    
    #7 point stencil
    k = [[[0,0,0],[0,1,0],[0,0,0]],[[0,1,0],[1,-6,1],[0,1,0]],[[0,0,0],[0,1,0],[0,0,0]]]
    """
    """
    
    L = convolve(M_padded, k)
    M_upd = L[2:-2,2:-2,2:-2]
    
    #L = convolve(M_padded, k, mode='same')
    #M_upd = L[1:-1,1:-1,1:-1]
    
    return M_upd


In [ ]:
def process_cell_nutrient_interactions(cells, S_list, shape_S, diff, dt, uptake_rates, release_rates):
    """
    Process nutrient uptake/release for all cells and update their states
    
    Parameters:
    cells: list of Cell objects
    S_list: list of nutrient concentration arrays
    shape_S: tuple of (nx, ny, nz) for the coarse grid
    diff: integer factor between fine and coarse grid
    dt: timestep size
    uptake_rates: array of uptake rates per nutrient
    release_rates: array of release rates per nutrient
    
    Returns:
    uptake_matrix: array of uptake amounts per grid cell per nutrient
    release_matrix: array of release amounts per grid cell per nutrient
    """
    n_nutrients = len(S_list)
    uptake_matrix = np.zeros((n_nutrients, *shape_S))
    release_matrix = np.zeros((n_nutrients, *shape_S))
    
    for cell in cells:
        # Convert fine grid position to coarse grid index
        i = int(cell.x // diff)
        j = int(cell.y // diff)
        k = int(cell.z // diff)
        
        # Skip if cell is outside bounds
        if not (0 <= i < shape_S[0] and 0 <= j < shape_S[1] and 0 <= k < shape_S[2]):
            continue
            
        for nutrient_idx in range(n_nutrients):
            local_concentration = S_list[nutrient_idx][i, j, k]
            
            # Process uptake
            if cell.dependencies[nutrient_idx, 0]:  # if cell uptakes this nutrient
                # Calculate actual uptake based on available nutrient
                desired_uptake = uptake_rates[nutrient_idx] * dt
                actual_uptake = min(desired_uptake, local_concentration)
                
                # Update tracking matrices
                uptake_matrix[nutrient_idx, i, j, k] += actual_uptake
                
                # Update cell's internal state
                cell.process_nutrient_uptake(nutrient_idx, actual_uptake)
            
            # Process release
            if cell.dependencies[nutrient_idx, 1]:  # if cell releases this nutrient
                release_amount = release_rates[nutrient_idx] * dt
                release_matrix[nutrient_idx, i, j, k] += release_amount
                
                # Update cell's internal state
                cell.process_nutrient_release(nutrient_idx, release_amount)
    
    return uptake_matrix, release_matrix

In [ ]:
def bud(cell):
    
    """
    The general strategy will be to generate a list of adjacent points with respect to periodic boundary 
    conditions. Then, make a smaller list of these points that are empty. If there are no empty points in the 
    plane, we will search the plane above. Once a list of potential points has been generated, the cell will 
    randomly bud into one. 
    """
    
    X = X_og.copy()
    
    #the coordinates of the cell to be propagated
    coords = np.array([x,y,z])
    
    shp = np.shape(X)
    
    
    #make a 3x3 stencil to search with
    xes = np.zeros(9)
    yes = np.zeros(9)

    xes[:3] = periodic_image(coords[0] - 1,shp[0])
    xes[3:6] = periodic_image(coords[0],shp[0])
    xes[6:] = periodic_image(coords[0] + 1,shp[0])

    yes[0::3] = periodic_image(coords[1] - 1,shp[1])
    yes[1::3] = periodic_image(coords[1],shp[1])
    yes[2::3] = periodic_image(coords[1] + 1,shp[1])
    
    
    poss_xvals = xes[X[xes.astype('int'),yes.astype('int'),int(coords[2])]==0]
    poss_yvals = yes[X[xes.astype('int'),yes.astype('int'),int(coords[2])]==0]
    
    n_possibilities = np.size(poss_xvals)
    floor = 0
    
    success = True

    
    if n_possibilities == 0:
        #print('trying one floor up')
        
        new_poss_xvals = xes[X[xes.astype('int'),yes.astype('int'),int(coords[2] + 1)]==0]
        new_poss_yvals = yes[X[xes.astype('int'),yes.astype('int'),int(coords[2] + 1)]==0]

        n_new_possibilities = np.size(new_poss_xvals)
        
        if n_new_possibilities == 0:
            #print('no available space, cannot bud')
            
            success = False
        else:
            poss_xvals = new_poss_xvals
            poss_yvals = new_poss_yvals
            n_possibilities = n_new_possibilities
            floor = 1
            
    if success:
        #print('Budding!')
        index_to_bud = int(np.floor(np.random.rand()*n_possibilities))

        X[int(poss_xvals[index_to_bud]),int(poss_yvals[index_to_bud]),int(coords[2]+floor)] = typ
        
    return X

In [ ]:
def update_nutrient_fields(S_list, D_list, cells, diff, dt, uptake_rates, release_rates):
    """
    Update nutrient concentration fields and cell states for one timestep
    
    Parameters:
    S_list: list of 3D nutrient concentration arrays
    D_list: list of diffusion coefficients for each nutrient
    cells: list of Cell objects
    diff: grid refinement factor
    dt: timestep size
    uptake_rates: array of uptake rates per nutrient
    release_rates: array of release rates per nutrient
    
    Returns:
    List of updated nutrient concentration arrays
    """
    n_nutrients = len(S_list)
    shape_S = S_list[0].shape
    S_new_list = []
    
    # Process all cell-nutrient interactions
    uptake_matrix, release_matrix = process_cell_nutrient_interactions(
        cells, S_list, shape_S, diff, dt, uptake_rates, release_rates
    )
    
    # Update each nutrient field
    for i in range(n_nutrients):
        # Compute diffusion term
        laplacian = convolution_laplacian_3d(S_list[i])
        diffusion_term = D_list[i] * laplacian
        
        # Combine diffusion with uptake/release
        source_sink_term = release_matrix[i] - uptake_matrix[i]
        
        # Forward Euler step
        S_new = S_list[i] + dt * (diffusion_term + source_sink_term)
        
        # Ensure non-negative concentrations
        S_new = np.maximum(S_new, 0)
        
        S_new_list.append(S_new)
    
    return S_new_list

In [ ]:
initial_cells = initialize_population(side_len,cell_size,n_strains,density,n_nutrients,dependencies,alphas,d)

begin_time = tm.time()

# Run parallel simulation
S_list_final, final_cells = parallel_simulate_system(
    [S1, S2], [D1, D2], initial_cells, cell_to_diff_ratio,
    dt_diff, dt_cell, dt_cell_to_diff_ratio, vm_list, km_list, release_rates,
    t_final
)


    



#the main loop
for tt in range(int(t_final/dt_cell)):
    for uu in range(dt_cell_to_diff_ratio):
        S1, S2 = update_nutrient_fields(
            [S1, S2], 
            [D1, D2], 
            cells, 
            diff, 
            dt,
            uptake_rates,
            release_rates
        )
        
end_time = tm.time()   
print('one %s'%t_final,' hour simulation took %s'%round(end_time-begin_time,0),'s')

print('Done! :D')